In [ ]:
%run '/content/drive/MyDrive/data_science_machine_learning/project_box_data_science/gral_functions/gral_funcs_box_data.ipynb'
import numpy as np


def box_data_merger_spliter(boxers_info_df, fights_info_df, fights_results_df):
  '''
  Function to merge the box data and return split for different purposes.
  Receives:
    boxers_info_df: Dataframe with boxers info.
    fights_info_df: Dataframe with fights info.
    fights_results_df: Dataframe with fights info.
  Returns:
    df: Datfarame with the columns the actual model needs.
    ids: Dataframe with the columns of id's.
    non_used_cols: Dataframe with the non used columns for the model.
  '''
  bo = boxers_info_df.copy()
  fi = fights_info_df.copy()
  fi['b1_bet'] = [1 + i if i < 1 else i for i in fi.b1_bet] # modifico valores menores que 0
  fi['b2_bet'] = [1 + i if i < 1 else i for i in fi.b2_bet]
  fr = fights_results_df.copy()
  df = fr.merge(fi, how='right', on='fight_id').\
  merge(bo, how='left', left_on='b1_id', right_on='id').drop(columns='id').\
  merge(bo, how='left', left_on='b2_id', right_on='id', suffixes=('_b1','_b2')).\
  drop(columns='id').rename(columns={'boxer_b1':'boxer1','boxer_b2':'boxer2','c_f':'date_f'})
  ids = df[['fight_id','b1_id','b2_id']]
  non_used_cols = df[['date_f',
              'reach_b1','boxstyle_b1','stance_b1',
              'text_boxer1','summary_boxer1','len_summary_boxer1',
              'box_live_data_b1',
              'reach_b2','boxstyle_b2','stance_b2',
              'text_boxer2','summary_boxer2','len_summary_boxer2',
              'box_live_data_b2']]
  df = df[['fight_id','result','endround','endtype','boxer1','boxer2',
          'birth_b1','height_b1',
          'b1_w','b1_wk','b1_d','b1_l','b1_lk','b1_bet',
          'len_text_boxer1',
          'birth_b2','height_b2',
          'b2_w','b2_wk','b2_d','b2_l','b2_lk','b2_bet',
          'len_text_boxer2']]
  return [df, ids, non_used_cols]


def data_column_sep(data):
  '''
  Function to separate the columns for info, fight between, X and y for the model.
  Receives:
    data: Dataframe to split.
  Returns:
    res: Df columns result, endround, endtype.
    pelea: Df columns boxer1, boxer2.
    X: Df columns nedeed for the model.
    y: Df column to predict.
  '''
  df = data.copy()
  df['result'] = df.result.replace({'1':1,'win':1,'w':1,'loss':0,'l':0,'0':0})
  df = df[df.result.isin([1,0,np.nan])]
  res = df[['result','endround','endtype']]
  pelea = df[['boxer1','boxer2']]
  X = df[['birth_b1','height_b1',
           'b1_w','b1_wk','b1_d','b1_l','b1_lk','b1_bet','len_text_boxer1',
           'birth_b2','height_b2',
           'b2_w','b2_wk','b2_d','b2_l','b2_lk','b2_bet','len_text_boxer2']]
  y = df['result']
  return res, pelea, X, y


def feat_eng_2(df):
  '''
  Feature engineering values from the 2nd (short) df.
  :df: Dataframe sobre el cual agregar las características.
  :return: df con las características.
  '''
  df = df.copy()
  suma_peleas_b1 = df.b1_w + df.b1_d + df.b1_l
  suma_peleas_b2 = df.b2_w + df.b2_d + df.b2_l
  df['b1_momios_menores'] = np.where(df.b1_bet < df.b2_bet, 1, 0)
  df['b1_bet_inversa'] = abs(1 - df.b1_bet)
  df['b2_bet_inversa'] = abs(1 - df.b2_bet)
  df = df.drop(columns=['b1_bet','b2_bet'])
  df['b1_mas_peleas'] = np.where(suma_peleas_b1 > suma_peleas_b2, 1, 0)
  df['b1_menos_peleas'] = np.where(suma_peleas_b1 < suma_peleas_b2, -1, 0)
  df['b1_mas_win_perc'] = np.where((df.b1_w / suma_peleas_b1) > (df.b2_w / suma_peleas_b2), 1, 0)
  df['b1_menos_win_perc'] = np.where((df.b1_w / suma_peleas_b1) < (df.b2_w / suma_peleas_b2), -1, 0)
  df['b1_menos_loss_perc'] = np.where((df.b1_l / suma_peleas_b1) < (df.b2_l / suma_peleas_b2), 1, 0)
  df['b1_mas_loss_perc'] = np.where((df.b1_l / suma_peleas_b1) > (df.b2_l / suma_peleas_b2), -1, 0)
  df['b1_mas_ko_perc'] = np.where((df.b1_wk / df.b1_w) > (df.b2_wk / df.b2_w), 1, 0)
  df['b1_menos_ko_perc'] = np.where((df.b1_wk / df.b1_w) < (df.b2_wk / df.b2_w), -1, 0)
  df['b1_invicto'] = np.where(df.b1_l <= 0, 1, 0)
  df['b2_invicto'] = np.where(df.b2_l <= 0, -1, 0)
  df['b1_more_fame'] = np.where(df.len_text_boxer1 > df.len_text_boxer2, 1, -1)
  return df


def train_func_cross_val(X, cluster_object, scaler=StandardScaler()):
  '''
  Perform data preprocessing and transformation on the input data for cross validation purposes.
    Parameters:
      :X: Input data (DataFrame or array-like).
      :cluster_object: A clustering model or object to transform the input data.
      :scaler: A feature scaling method (default is StandardScaler) for scaling the transformed data.
    Returns:
      :x_clus: Transformed data after clustering.
      :x_fe: Transformed data after feature engineering.
      :x_scal: Scaled and preprocessed data.
      :[cluster, scaler]: List with the cluster and scaler fitted.
  '''
  x_clus = cluster_object.fit_transform(X)
  x_fe = feat_eng_2(x_clus)
  x_scal = scaler.fit_transform(x_fe)
  return x_clus, x_fe, x_scal, [cluster_object, scaler]


def model_trainer(X_train, y_train, cluster_object, model_object, scaler=StandardScaler()):
  '''
  Train a machine learning model (Model_applied object) after data preprocessing.
  This function takes the training data, performs data preprocessing steps, including clustering and feature engineering,
  and then trains a machine learning model in a Model_applied object on the preprocessed data.
    Parameters:
      :X_train: Training data (DataFrame or array-like).
      :y_train: Target labels for the training data.
      :cluster: A Data_clusterer initialized object (it will be fitted and transform data).
      :model: The Model_applied initialized object to be trained.
      :scaler: A feature scaling method (default is StandardScaler) for scaling the preprocessed data.
    Returns:
      :x_clus: Transformed data after clustering.
      :x_fe: Transformed data after feature engineering.
      :x_scal: Scaled and preprocessed data.
      :[cluster, scaler, model]: A list containing the trained clustering model, scaler, and the trained machine learning model.
  '''
  x_clus, x_fe, x_scal, [cluster, scaler] = train_func_cross_val(X_train, cluster_object, scaler)
  model_object.fit(x_scal, y_train)
  explainer = LimeTabularExplainer(x_scal, training_labels=y_train, mode="classification", feature_names=x_fe.columns)
  return x_clus, x_fe, x_scal, [cluster_object, scaler, model_object, explainer]


def new_preds(X, fitted_cluster, fitted_scaler, fitted_model):
  '''
  Function that join the steps to bring a prediction for new data since before feature engineering process, without encoder.
  Parameters:
    :X: First data to predict
    :feature_eng_func: Function to feature engineering.
    :fitted_cluster: Object of class Data_clusterer() initialized and fitted.
    :fitted_scaler: Object of class StandardScaler() [or other scaler] initialized and fitted.
    :fitted_model: Object of class Model_applied() initialized and fitted.
  Returns:
    DataFrame with columns ['boxer1_pred', 'prob_win', 'prob_loss', 'cluster', 'initial_index'].
  '''
  x_clus = fitted_cluster.transform(X)
  x_fe = feat_eng_2(x_clus)
  x_scal = fitted_scaler.transform(x_fe)
  y_pred = fitted_model.transform(x_scal)
  y_pred['initial_index'] = x_clus.index
  y_pred = y_pred.merge(x_clus.cluster, how='left', left_on='initial_index', right_index=True)
  return x_clus, x_fe, x_scal, y_pred

def check_predictions(results_df, fight_df, preds_df, ids_df, cluster_df):
  '''
  Function that returns merged df's in order to analyse model predictions.
  Parameters:
    results_df: Dataframe with the results of the fights predicted.
    fight_df: Dataframe with the fights predicted.
    pred_df: Dataframe with the predictions of the fights.
    ids_df: Dataframe with the id's of the fights predicted.
    clusters_df: Dataframe with the cluster column of the fights predicted.
  Returns:
    df_basic: Dataframe with most important columns to analyse.
    df_ids: Dataframe with the id columns.
    df_full: Dataframe with all the merged columns.
  '''
  df_basic = results_df.merge(fight_df, how='right', left_index=True, right_index=True).merge(\
                           preds_df, how='left', left_index=True, right_on='initial_index')
  df_ids = df_basic.merge(ids_df, how='left', left_on='initial_index', right_index=True)
  df_full = df_ids.merge(cluster_df, how='left', left_on='initial_index', right_index=True)
  df_full['goodpred'] = ['' if pd.isna(i) else i == e for i,e in zip(df_full.result, df_full.boxer1_pred)]
  df_full.rename(columns={'cluster_x':'cluster'}, inplace=True)
  df_basic = df_full[['initial_index', 'goodpred', 'prob_win','cluster',
                       'boxer1_pred', 'result', 'b1_bet', 'b2_bet',
                       'boxer1', 'boxer2', 'endround', 'endtype']]
  df_ids = df_full[['fight_id', 'result', 'endround', 'endtype',
                  'b1_id', 'boxer1', 'b2_id', 'boxer2', 'cluster',
                  'boxer1_pred', 'prob_win', 'initial_index']]
  df_full = df_full[['fight_id', 'result', 'endround', 'endtype',
     'cluster', 'boxer1_pred', 'goodpred', 'prob_win', 'initial_index',
     'b1_id', 'boxer1', 'birth_b1', 'height_b1', 'b1_w',
       'b1_wk', 'b1_d', 'b1_l', 'b1_lk', 'b1_bet', 'len_text_boxer1',
     'b2_id', 'boxer2', 'birth_b2', 'height_b2', 'b2_w',
       'b2_wk', 'b2_d', 'b2_l', 'b2_lk', 'b2_bet', 'len_text_boxer2']]
  return df_basic, df_ids, df_full